- This script compares how well the neural responses to social content correlates with internalizing symptom scores, i.e., the trait-behavior-fMRI relationship
- Renaming notebook for clarity (former name: 2_3_3_fMRI_SocNonsoc_v_SocNonsocwTraits_AIC_nooutput.ipynb)

Rekha Varrier, July-August 2022

In [ ]:
# import general packages, check folders
#%reset
import os
import numpy as np
import pandas as pd
#import imagesc as imagesc #pip install imagesc
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import time
from pathlib import Path

%matplotlib inline

working_dir = '/Users/f0053cz/Dropbox (Dartmouth College)/postdoc_Dartmouth/HCP/fMRIScipts/code'
#working_dir = os.getcwd()
print('current directory:\n',working_dir)
path = Path(working_dir)
parent_folder = path.parent
data_file_loc = os.path.join(parent_folder,'data') # to store data we extract later in this notebook

In [ ]:
# setting default fontsizes for plots

s=20 # CHANGE FONTSIZE HERE
plt.rc('font', size=s) #controls default text size
plt.rc('axes', titlesize=s) #fontsize of the title
plt.rc('axes', labelsize=s) #fontsize of the x and y labels
plt.rc('xtick', labelsize=s) #fontsize of the x tick labels
plt.rc('ytick', labelsize=s) #fontsize of the y tick labels
plt.rc('legend', fontsize=s) #fontsize of the legend

In [ ]:
#suff= ' '
suff = '_corrected'

In [ ]:
# load behavioral data - coded 1 for "social", 0 for "nonsocial" and 9 for "unsure", nan for missed response
# even if using the Mental/Random labels, need this to sub-select subs who have responded on all trials
responses = np.load(os.path.join(data_file_loc,f'responses{suff}.npy'))
responses.shape # subs *movies

In [ ]:
print('Number of complete subjects:',len(np.where(np.array([len(np.where(~np.isnan(responses[s,:]))[0]) for s in range(responses.shape[0])])==10)[0]))
print('Total subjects:', responses.shape[0])

In [ ]:
# timepts_indiv_movie = file with TRs within run to be selected for each movie (including a few fixation TRs at the start and end),
# vid_start_rel_tr = the TR at which the movie begins within each timecourse snippet
# details in 1_1_create_regressors.ipynb
[timepts_indiv_movie,vid_start_rel_tr] = np.load(os.path.join(data_file_loc,'Video_TRs.npy'),allow_pickle=True)
vid_start_rel_tr

# Run-level regressors vs. traits 

In [ ]:
# import run-level regression coefficients (saved in 2_2_MRI_LM_allmovies_generateregcoeffts.ipynb)
flName = os.path.join(data_file_loc,f'coef_slopereg_all{suff}.npy')
run_level_coefs = np.load(flName) # dimensions: subject *  brain parcels * runs

run_level_coefs_mean = np.nanmean(run_level_coefs, axis=2) # 1 coefft per run, average across runs
run_level_coefs_mean.shape 

In [ ]:
# packages for linear regression and multiple comparison
from pymer4.models import
from multipy.fdr import lsu
from datetime import datetime

# social processing regions across the various GLM analyses - highlighted in the first S>NS GLM as black contours. Using it for contours here too
nodes_coaxbill_rand_all = np.load(os.path.join(data_file_loc,'nodes_coaxbill_rand_all.npy')) 

In [ ]:
data_file_loc = '/Users/f0053cz/Dropbox (Dartmouth College)/postdoc_Dartmouth/HCP/fMRIScipts/data'
sub_id_all = np.load(os.path.join(data_file_loc,f'sub_ID_all{suff}.npy')) # subject IDs assigned by HCP for comparison with trait scores
sub_id_all = [str(i) for i in sub_id_all]
sub_id_all = np.array(sub_id_all)
len(sub_id_all)

In [ ]:
# names of all the animations in the presented order (they were never randomized)
vidnames = ["COAXING-B", "BILLIARD-A", "DRIFTING-A", "Fishing", "Random mechanical","Scaring", "SEDUCING-B", "STAR-A", "SURPRISING-B", "TENNIS-A"]

# creating a 3D array of beta coeffs across movies from individual movie files
all_coefs = np.zeros((responses.shape[0],268,10)) # subject * nodes * movies (i.e., stimuli)

for m in range(10): # 10 movies
    fileName =  os.path.join(data_file_loc,'coefs_run_norm','slope_reg',f'coef_slopereg_runnorm_{vidnames[m]}{suff}.npy')
    all_coefs[:,:,m] = np.load(fileName)

all_coefs[:10,0,:2]

In [ ]:
# best to remove subjects with < 10 responses (i.e., any missing responses) for power in general
count_resp = np.zeros((responses.shape[0],))
for i in range(responses.shape[0]):
    count_resp[i] = len(np.where(~np.isnan(responses[i,:]))[0])
subs_10resp = np.where(count_resp == 10)[0]
len(subs_10resp)

In [ ]:
res_behav_data = pd.read_csv('/Users/f0053cz/Documents/HCP_trait_diffs_DONOTSHARE/trait_analyses/data/RESTRICTED_esfinn_11_21_2021_19_19_35.csv')
res_behav_data.set_index("Subject", inplace=True)
res_behav_data.index = res_behav_data.index.map(str)
print(res_behav_data.shape)

# Effect of "Social" - "Non-social" beta and trait scores (without considering movies)

- Fitting each run-level regression coefficient (which already summarizes S-NS) with the internalizing trait score (ASR_Intn_T) for each brain region across people

In [ ]:

model9_txt = 'coefs_diff ~ ASR_Intn_T'
nsubs_node = []

# LM - run-level regs
model9_res = []
rval_pval_spearman_coef_diff_ASR_Intn_T = []
start_time =  time.time()

for n in range(268): # all brain regions/nodes/parcesl
    if n%10 == 0: # print status
        print(f'node {n+1},time={(time.time()-start_time)/60:.2f} mins')
    
    df = pd.DataFrame({'Subject':sub_id_all[subs_10resp], 'coefs_diff': run_level_coefs_mean[subs_10resp,n], 'subID':subs_10resp})
    df.set_index("Subject", inplace=True) # save fMRI run-level data as a dataframe

    df_both = df.join(res_behav_data.loc[:,['ASR_Intn_T']], how='inner') # join the run-level regression coefft (i.e., beta S -NS)
    #  and trait info
    inds = ~np.isnan(df_both['coefs_diff']) & ~np.isnan(df_both['ASR_Intn_T']) # find rows where neither x or y is NaN
    nsubs_node.append(len(np.where(inds)[0])) # store the non-NaN subs for each node in this list (sanity check for later if needed)
    df_both = df_both.loc[inds,:] # df without NaNs
    
     # linear regressionm setting up and fititng model
    model9 = Lm(model9_txt, data=df_both)
    model9.fit(summary = False, verbose = False)
    model9_res.append([model9.coefs['Estimate'],model9.coefs['P-val'],model9.AIC]) 
    # store results of the model - i.e., the Estimate for traits, its p-value and AIC, which is useful to compare against other models

    # parallelly, also doing a correlation between the two. should give somewhat similar results as the above, but less precise.
    rs,ps = stats.spearmanr(df_both['coefs_diff'],df_both['ASR_Intn_T']) 
    rval_pval_spearman_coef_diff_ASR_Intn_T.append([rs,ps])
    if n == 0:
        print('model:',model9.fit(summary = True, verbose = False))

rval_pval_spearman_coef_diff_ASR_Intn_T = np.array(rval_pval_spearman_coef_diff_ASR_Intn_T)
print('Done on/at:',datetime.now()) # cell run at)

In [ ]:
# extract the estimate and p-value corresponding to the trait term
# The coefficient for the trait term (ASR_Intn_T) is an interaction-like term, since it summarizes how much this trait covaries with a 
# difference term for brain activity between social and non-social
print(model9_txt)
coef_p_model9 = [[model9_res[n][0][1], model9_res[n][1][1]] for n in range(268)] # ASR reg coefft
coef_p_model9= np.array(coef_p_model9)
print(coef_p_model9[:5,:])

In [ ]:
print("Traits term's dependence on ASR_Intn_T:")
print('nodes with p < .05 :', np.where(coef_p_model9[:,1]<.05)[0])

coef_p_model9_fdr = lsu(coef_p_model9[:,1],q=.05)#fdr_correction(pval_slope_rand,.05)
print('nodes with q<.05:', np.where(coef_p_model9_fdr)[0])

min(coef_p_model9[:,0]),max(coef_p_model9[:,0])

# Plot results on the brain

- Some of the packages and funcitons have also been used in 2_3_1.. and 2_3_2..., so same idea here too
    

In [ ]:
from nilearn.surface import vol_to_surf
from nilearn.plotting import plot_glass_brain, plot_surf_roi,plot_stat_map,plot_img,plot_surf_contours
import nibabel as nib
from nilearn import datasets
bg_img = datasets.load_mni152_template()
fig_save_loc = os.path.join('/Users/f0053cz/Dropbox (Dartmouth College)/postdoc_Dartmouth/HCP/paper_prep/figures/fig2_2_traits/fmri_results/ALL')

from nilearn.datasets import fetch_surf_fsaverage
fsaverage = fetch_surf_fsaverage()

from nilearn import datasets
bg_img = datasets.load_mni152_template()

from nltools.data import Brain_Data
from nltools.mask import expand_mask, roi_to_brain


mask = Brain_Data('https://neurovault.org/media/images/8423/shen_2mm_268_parcellation.nii.gz')
mask_x = expand_mask(mask)

def color_rois(values):
    """
    This function assumes you are passing a vector "values" with the same length as the number of nodes in the atlas.
    """

    shen268 = nib.load(os.path.join(data_file_loc,"shen_2mm_268_parcellation.nii.gz"))
    shen268_data = shen268.get_fdata()
    img = np.zeros(shen268_data.shape)
    #print(shen268_data.shape)
    for roi in range(len(values)):
        itemindex = np.where(shen268_data==roi+1) # find voxels in this node (add 1 to account for zero-indexing)
        #print(len(itemindex[0]))
        img[itemindex] = values[roi] # color them by the desired value 
    affine = shen268.affine
    img_nii = nib.Nifti1Image(img, affine)
    return img_nii

def surf_plot1(fig,ax,nodes,params, thresh):
    title_txt = params['title']
    txt  = params['txt']
    vmin = params['vmin']
    vmax = params['vmax']

    # LH (left hemisphere)
    ax_surf = ax[0,0] # lateral
    texture = vol_to_surf(color_rois(nodes), fsaverage.pial_left,interpolation='nearest',radius =1, n_samples=1)
    surf_plot1=plot_surf_roi(fsaverage.infl_left, texture, hemi='left',cmap = cmap, colorbar=False, symmetric_cmap=True, vmin = vmin, vmax = vmax,
                                bg_map=fsaverage.sulc_left,axes = ax_surf, threshold =  thresh)#)#,vol_to_surf_kwargs={"n_samples": 10, "radius": 10, "interpolation": "nearest","kind":'ball'})
    texture_contour = vol_to_surf(color_rois(nodes_coaxbill_rand_all), fsaverage.pial_left,interpolation='nearest',radius =1, n_samples=1)
    plot_surf_contours(fsaverage.infl_left, texture_contour, axes = ax_surf,figure=surf_plot1, legend=True,levels = [1], colors=['k'])

    ax_surf = ax[1,0] # medial
    surf_plot2=plot_surf_roi(fsaverage.infl_left, texture, hemi='left',cmap = cmap, colorbar=False, symmetric_cmap=True, vmin = vmin, vmax = vmax,
                                bg_map=fsaverage.sulc_left, view = 'medial',axes = ax_surf, threshold =  thresh)#,vol_to_surf_kwargs={"n_samples": 10, "radius": 10, "interpolation": "nearest","kind":'ball'})
    plot_surf_contours(fsaverage.infl_left, texture_contour, axes = ax_surf,figure=surf_plot2, legend=True,levels = [1], colors=['k'])

    # RH (right hemisphere)
    ax_surf = ax[0,1] # lateral
    texture = vol_to_surf(color_rois(nodes), fsaverage.pial_right,interpolation='nearest',radius =1, n_samples=1)
    surf_plot3=plot_surf_roi(fsaverage.infl_right, texture, hemi='right',cmap = cmap, colorbar=True,symmetric_cmap=True, vmin = vmin, vmax = vmax,
                                bg_map=fsaverage.sulc_right,axes = ax_surf, threshold =  thresh)#)#,vol_to_surf_kwargs={"n_samples": 10, "radius": 10, "interpolation": "nearest","kind":'ball'})
    texture_contour = vol_to_surf(color_rois(nodes_coaxbill_rand_all), fsaverage.pial_right,interpolation='nearest',radius =1, n_samples=1)
    plot_surf_contours(fsaverage.infl_right, texture_contour, axes = ax_surf,figure=surf_plot3, legend=True,levels = [1], colors=['k'])
    surf_plot3.axes[4].text(10,.5*vmax,s=txt,fontsize=20, fontdict = {'verticalalignment':'top','horizontalalignment':'left','rotation':0})
    box = surf_plot3.axes[4].get_position()
    surf_plot3.axes[4].set_position([box.x0*.93, box.y0-.3, box.width, box.height*2])  # move a bit the bar to the right, need to divide by number of columns (to move relative to last figure only, not to overall row, else will get too far away)
    

    ax_surf = ax[1,1] # medial
    surf_plot4=plot_surf_roi(fsaverage.infl_right, texture, hemi='right',cmap = cmap, colorbar=False,symmetric_cmap=True, vmin = vmin, vmax = vmax,
                                bg_map=fsaverage.sulc_right, view ='medial',axes = ax_surf, threshold =  thresh)#)#,vol_to_surf_kwargs={"n_samples": 10, "radius": 10, "interpolation": "nearest","kind":'ball'})
    plot_surf_contours(fsaverage.infl_right, texture_contour, axes = ax_surf,figure=surf_plot4, legend=True, levels = [1], colors=['k'])#, labels=['Sig. (q<.05) across\n(a),(c),(d)'])

    ax[0,0].dist = 7 # change viewing distance to "zoom in" to surface plots
    ax[0,1].dist = 7
    ax[1,0].dist = 7
    ax[1,1].dist = 7

    #fig.colorbar(surf_plot3.axes[2])
    plt.subplots_adjust(left=0,
                        bottom=0, 
                        right=.8, 
                        top=1, 
                        wspace=0.0, 
                        hspace=-.1)    
fig_save_loc

## All nodes that are significant highlighted, with GLM contours to see which ones are within the social processing regions and which ones are not

- Figure 7e in the final manuscript

In [ ]:
# surface plots
vmin,vmax = -.003,.003
cmap =  'RdBu_r' #'RdBu_r', 'PiYG'
txt = '     'r"$\overline{\beta}(traits)$" # to be corrected - remove overline (NOT "MEAN" BETA)
title_txt = 'traits term beta (p<.05)'
 
thresh = 1e-10
nodes = np.zeros((268,))
nodes[(coef_p_model9[:,1]<.05)] = coef_p_model9[(coef_p_model9[:,1]<.05),0]

fig,ax = plt.subplots(nrows=2, ncols= 2,figsize=(6,4),subplot_kw={'projection': '3d'})
params = {'title':title_txt,'txt':txt, 'vmin': vmin, 'vmax':vmax}
surf_plot1(fig,ax,nodes,params,thresh = 1e-10)
plt.savefig(os.path.join(fig_save_loc,f'soc_vs_nonsoc/resp_trait_corr_unc_all_nodes_surf.png'),dpi=300,bbox_inches='tight',facecolor='white', edgecolor='none')

img = roi_to_brain(pd.Series(nodes), mask_x)
coords = [-44,-34,-24] # initial exploration
ax_plot = plot_img(img.to_nifti(), display_mode = 'z',vmin = vmin, vmax = vmax, cut_coords =coords,cmap = cmap, bg_img = bg_img, threshold=thresh,
colorbar= False)
ax_plot.add_contours(color_rois(nodes_coaxbill_rand_all),linewidths=1, colors=['k'],linestyles ='-',filled=False)

plt.savefig(os.path.join(fig_save_loc,f'soc_vs_nonsoc/resp_trait_corr_unc_all_nodes_axial.png'),dpi=300,bbox_inches='tight',facecolor='white', edgecolor='none')
#plt.clf()

## Not in paper

### Filtering nodes by whether they're also within the intersection social processing nodes

In [ ]:
# surface plots, 
vmin,vmax = -.003,.003
cmap =  'RdBu_r' #'RdBu_r', 'PiYG'
txt = '     'r"$\overline{\beta}(traits)$" # to be corrected - remove overline (NOT "MEAN" BETA)
title_txt = 'traits term beta (p<.05)'
 
thresh = 1e-10
nodes = np.zeros((268,))
nodes[(coef_p_model9[:,1]<.05) &  (nodes_coaxbill_rand_all)] = coef_p_model9[(coef_p_model9[:,1]<.05) & (nodes_coaxbill_rand_all),0]

fig,ax = plt.subplots(nrows=2, ncols= 2,figsize=(6,4),subplot_kw={'projection': '3d'})
params = {'title':title_txt,'txt':txt, 'vmin': vmin, 'vmax':vmax}
surf_plot1(fig,ax,nodes,params,thresh = 1e-10)
plt.savefig(os.path.join(fig_save_loc,f'soc_vs_nonsoc/resp_trait_corr_unc_GLM nodes_surf.png'),dpi=300,bbox_inches='tight',facecolor='white', edgecolor='none')

img = roi_to_brain(pd.Series(nodes), mask_x)
coords = [-44,-34,-24] # initial exploration
ax_plot = plot_img(img.to_nifti(), display_mode = 'z',vmin = vmin, vmax = vmax, cut_coords =coords,cmap = cmap, bg_img = bg_img, threshold=thresh,
colorbar= False)
ax_plot.add_contours(color_rois(nodes_coaxbill_rand_all),linewidths=1, colors=['k'],linestyles ='-',filled=False)

plt.savefig(os.path.join(fig_save_loc,f'soc_vs_nonsoc/resp_trait_corr_unc_GLM nodes_axial.png'),dpi=300,bbox_inches='tight',facecolor='white', edgecolor='none')
#plt.clf()

### Correlation (as an alternative to linear regression) between run-wise diff and traits

- this would be less sensitive to magnitudes, but doesn't have an intercept term like linear regression

In [ ]:
print(np.where(rval_pval_spearman_coef_diff_ASR_Intn_T[:,1]<.05))
print(np.where(lsu(rval_pval_spearman_coef_diff_ASR_Intn_T[:,1])))
print(np.where(lsu(rval_pval_spearman_coef_diff_ASR_Intn_T[nodes_coaxbill_rand_all,1])))

#### Plotting results from the correlation 

In [ ]:
qcorr = np.zeros((268,))
qcorr[nodes_coaxbill_rand_all] = lsu(rval_pval_spearman_coef_diff_ASR_Intn_T[nodes_coaxbill_rand_all,1])
np.where(qcorr)

# nodes showing sig (unc p <.05) correlation between "Social"-"Non-social" and trait score
vmin,vmax = -.08,.08
cmap =  'RdBu_r' #'RdBu_r', 'PiYG'
#txt = r"$\overline{\beta}(''S''-''NS'')$"
txt = "rSp_traits" 

title_txt = 'traits term Spearman_r (p<.05)' 
thresh = 1e-10

nodes = np.zeros((268,))
nodes[rval_pval_spearman_coef_diff_ASR_Intn_T[:,1]<.05] = rval_pval_spearman_coef_diff_ASR_Intn_T[rval_pval_spearman_coef_diff_ASR_Intn_T[:,1]<.05,0]

fig,ax = plt.subplots(nrows=2, ncols= 2,figsize=(6,4),subplot_kw={'projection': '3d'})
params = {'title':title_txt,'txt':txt, 'vmin': vmin, 'vmax':vmax}
surf_plot1(fig,ax,nodes,params,thresh = 1e-10)
#plt.savefig(os.path.join(fig_save_loc,f'AIC_surf_thresh{thresh}_resp_resptraits.png'),dpi=300,bbox_inches='tight',facecolor='white', edgecolor='none')

img = roi_to_brain(pd.Series(nodes), mask_x)
coords = [-44,-34,-24] # initial exploration
ax_plot = plot_img(img.to_nifti(), display_mode = 'z',vmin = vmin, vmax = vmax, cut_coords =coords,cmap = cmap, bg_img = bg_img, threshold=thresh,
colorbar= False)
ax_plot.add_contours(color_rois(nodes_coaxbill_rand_all),linewidths=1, colors=['k'],linestyles ='-',filled=False)



print(model9_txt)
coef_p_model9 = [[model9_res[n][0][1], model9_res[n][1][1]] for n in range(268)] # ASR reg coefft
coef_p_model9= np.array(coef_p_model9)
print(coef_p_model9[:5,:])

print('sig terms:', len(np.where(coef_p_model9[:,1]<.05)[0]))
#coef_p_model9[coef_p_model9[:,1]<.05,1]
coef_p_model9_fdr = lsu(coef_p_model9[:,1],q=.05)#fdr_correction(pval_slope_rand,.05)
len(np.where(coef_p_model9_fdr)[0])

min(coef_p_model9[:,0]),max(coef_p_model9[:,0])

# surface plots
vmin,vmax = -.003,.003
cmap =  'RdBu_r' #'RdBu_r', 'PiYG'
txt = r"$\overline{\beta}(traits)$"

title_txt = 'traits term beta (p<.05)'
thresh = 1e-10

nodes = np.zeros((268,))
nodes[coef_p_model9[:,1]<.05] = coef_p_model9[coef_p_model9[:,1]<.05,0]

fig,ax = plt.subplots(nrows=2, ncols= 2,figsize=(6,4),subplot_kw={'projection': '3d'})
params = {'title':title_txt,'txt':txt, 'vmin': vmin, 'vmax':vmax}
surf_plot1(fig,ax,nodes,params,thresh = 1e-10)

img = roi_to_brain(pd.Series(nodes), mask_x)
coords = [-44,-34,-24] # initial exploration
ax_plot = plot_img(img.to_nifti(), display_mode = 'z',vmin = vmin, vmax = vmax, cut_coords =coords,cmap = cmap, bg_img = bg_img, threshold=thresh,
colorbar= False)
ax_plot.add_contours(color_rois(nodes_coaxbill_rand_all),linewidths=1, colors=['k'],linestyles ='-',filled=False)